In [11]:
import torch
from torch import nn
from torchvision import models, datasets, transforms
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
import time
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import os

class SEBlock(nn.Module):
    def __init__(self, in_channels, r=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1, 1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // r),
            nn.ReLU(),
            nn.Linear(in_channels // r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        out = self.squeeze(x).view(b, c)
        out = self.excitation(out).view(b, c, 1, 1)
        out = x * out.expand_as(x)
        return out


class DenseNetCustom(nn.Module):
    def __init__(self, num_classes):
        super(DenseNetCustom, self).__init__()
        self.base_model = models.densenet169(pretrained=True)
        self.features = self.base_model.features
        self.gradients = None
        self.activations = None
        
        # Adding SE-Block after the last 3 denseblock layers
        self.se_block1 = SEBlock(128)
        self.se_block2 = SEBlock(256)
        self.se_block3 = SEBlock(1664)

        # Classifier
        num_ftrs = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Linear(num_ftrs, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1664, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.se_block1(x)
        x = self.se_block2(x)
        x = self.se_block3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def get_activations_gradient(self):
        return self.gradients

    def get_activations(self, x):
        return self.features(x)

    def activations_hook(self, grad):
        self.gradients = grad



In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [13]:
import torchvision
# Load data
data_dir = '/home/kevinluo/breast_density_classification/INbreast'

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.Resize((640,640)),
                                       transforms.ToTensor()])

valid_transforms = transforms.Compose([transforms.Resize((640,640)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.Resize((640,640)),
                                      transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=4,shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=4)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4)

dataloaders = {
    'train': trainloader,
    'val': validloader
}

dataset_sizes = {
    'train': len(train_data),
    'val': len(valid_data)
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = DenseNetCustom(num_classes=4)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

# Generate heatmap
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    heatmap = model_ft.get_activations_gradient().cpu().data.numpy()
    fig, ax = plt.subplots()
    im = ax.imshow(heatmap)
    plt.colorbar(im)
    plt.show()


Epoch 0/0
----------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x1664 and 128x8)